In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import MetaStrategy.common_perf_ana as cpa

In [ ]:
def download_price_data(tickers, start_date, end_date):
    # Download historical price data for the tickers https://github.com/ranaroussi/yfinance/blob/f08fe83290136d103d46d67524f5b6e7b6b827ff/yfinance/utils.py
    # data = yf.download(tickers, start=start_date, end=pd.to_datetime(end_date) + pd.DateOffset(days= 1), interval="1d", back_adjust=True)
    # adjCloses = yf.download(tickers, start=start_date, end=pd.to_datetime(end_date) + pd.DateOffset(days= 1), interval="1d")['Adj Close']

    data = yf.download(tickers, start=start_date, end=end_date, interval="1d", back_adjust=True)
    adjCloses = yf.download(tickers, start=start_date, end=end_date, interval="1d")['Adj Close']

    adjHighs = data['High']
    adjLows = data['Low']
    adjOpens = data['Open']
    return adjCloses, adjHighs, adjLows, adjOpens

def calculate_rrg(benchmark, adjCloses, start_date, end_date):
    # Calculate RRG logic here
    # This can involve comparing the performance of each ETF to the benchmark

    # Download historical price data for benchmark
    benchmarkAdjClose = yf.download(benchmark, start=start_date, end=end_date)['Adj Close']
    
    # Calculate relative prices
    relPrices = adjCloses.div(benchmarkAdjClose, axis = 0)
    
    # Calculate 10-day and 30-day moving averages
    relPrices10MA = relPrices.rolling(window=10).mean()
    relPrices30MA = relPrices.rolling(window=30).mean()

    # Calculate relative strength, the ratio of 10-day and 30-day moving averages of relative prices
    relStrength = relPrices10MA / relPrices30MA * 100

    # Calculate relative momentum, the ratio of 1-day and 9-day moving averages of relative strength
    relMom = relStrength / relStrength.rolling(window=9).mean() * 100

    return relStrength, relMom

def calculate_rrg_Quadrant(relStrength, relMom):
    # Leading: RS >= 100, RM >= 100     Code: 1
    # Weakening: RS >= 100, RM < 100    Code: 2
    # Lagging: RS < 100, RM < 100       Code: 3
    # Improving: RS < 100, RM >= 100     Code: 4

    Quadrant_df = pd.DataFrame(index=relStrength.index, columns=relStrength.columns)

    for ticker in relStrength.columns:
        rs = relStrength[ticker]
        rm = relMom[ticker]

        Quadrant_df[ticker] = 0  # Initialize to 0

        # Update Quadrant based on conditions
        Quadrant_df.loc[(rs >= 100) & (rm >= 100), ticker] = 1  # Leading
        Quadrant_df.loc[(rs >= 100) & (rm < 100), ticker] = 2  # Weakening
        Quadrant_df.loc[(rs < 100) & (rm < 100), ticker] = 3  # Lagging
        Quadrant_df.loc[(rs < 100) & (rm >= 100), ticker] = 4  # Improving

    return Quadrant_df

def calculate_atr(close_df, high_df, low_df, atr_length):
    # Initializing DataFrames
    atr_df = pd.DataFrame(index=close_df.index)

    for ticker in close_df.columns:
        # Calculating True Range (TR)
        high_low_diff = high_df[ticker] - low_df[ticker]
        high_close_diff = abs(high_df[ticker] - close_df[ticker].shift(1))
        low_close_diff = abs(low_df[ticker] - close_df[ticker].shift(1))
        true_range = pd.concat([high_low_diff, high_close_diff, low_close_diff], axis=1).max(axis=1)

        # Calculating ATR
        atr_df[ticker] = true_range.rolling(window=atr_length).mean()

    return atr_df

def calculate_atr_stop_loss_price(close_df, atr_df, atr_length, stop_loss_multiplicator, stop_loss_barrier_type):
    # Initializing DataFrames
    atr_stop_df = pd.DataFrame(index=close_df.index)

    for ticker in close_df.columns:
        atr_stop_df[ticker] = close_df[ticker].rolling(window = atr_length + 1).max(axis=0) - atr_df[ticker].rolling(window = 1 + (atr_length - 1) * stop_loss_barrier_type ).max(axis=0) * stop_loss_multiplicator

    return atr_stop_df

def calculate_price_to_stop_ratio_df(adj_close_df, stop_price_df):
    # Initializing DataFrame
    ratio_df = pd.DataFrame(index=adj_close_df.index, columns=adj_close_df.columns)

    for ticker in adj_close_df.columns:
        prev_stop_prices = stop_price_df[ticker].shift(1)
        ratio_df[ticker] = adj_close_df[ticker] / prev_stop_prices - 1

    return ratio_df

def calculate_distance_from_origo_df(relStrength, relMom):
    # Initializing DataFrame
    distance_df = pd.DataFrame(index=relStrength.index, columns=relStrength.columns)

    for ticker in relStrength.columns:
        distance_df[ticker] = np.sqrt((relStrength[ticker] - 100)**2 + (relMom[ticker] - 100)**2)

    return distance_df

def manage_portfolio(initial_pv, no_etfs_selected, Quadrant_df, distance_df, adj_close_df, adj_open_df, price_to_stop_df, tieDecider, sellTypeQ, sellTypeSl, selected_rrg_Quadrant, exit_rrg_Quadrant, trade_type):
    # Initialize variables
    portfolio_df = pd.DataFrame(index=Quadrant_df.index, columns=['Cash', 'Portfolio Value'])
    portfolio_df.iloc[0]['Cash'] = initial_pv
    portfolio_df.iloc[0]['Portfolio Value'] = initial_pv
    no_played_etfs = 0
    position = pd.DataFrame(index=Quadrant_df.index, columns=Quadrant_df.columns)

    # Iterate through days
    for i in range(1, len(Quadrant_df)):
        date = Quadrant_df.index[i]

        # Copy positions from the previous day
        position.iloc[i] = position.iloc[i-1]

        # Calculate the current portfolio value
        position_value = (position.iloc[i] * adj_close_df.iloc[i]).sum()

        # Copy cash from the previous day
        portfolio_df.loc[date, 'Cash'] = portfolio_df.iloc[i-1]['Cash']

        # Update Portfolio Value
        portfolio_df.loc[date, 'Portfolio Value'] = portfolio_df.loc[date, 'Cash'] + position_value

        # Sell logic
        for ticker in position.columns:
            if pd.isna(position.loc[date, ticker]):
                continue
            if trade_type == 'OPG' and (i < (len(Quadrant_df) - 1)):
                sell_price = adj_open_df.iloc[i+1][ticker]
            else:
                sell_price = adj_close_df.iloc[i][ticker] 

            if sellTypeSl == 1 and price_to_stop_df.loc[date, ticker] < 0:
                # Sell the entire position
                portfolio_df.loc[date, 'Cash'] += position.loc[date, ticker] * sell_price
                no_played_etfs -= 1
                position.loc[date, ticker] = np.nan
            elif sellTypeQ == 'OutOfQuadrant' and Quadrant_df.loc[date, ticker] != selected_rrg_Quadrant:
                # Sell the entire position
                portfolio_df.loc[date, 'Cash'] += position.loc[date, ticker] * sell_price
                no_played_etfs -= 1
                position.loc[date, ticker] = np.nan
            elif sellTypeQ == 'ReachSellQuadrant' and Quadrant_df.loc[date, ticker] == exit_rrg_Quadrant:
                # Sell the entire position
                portfolio_df.loc[date, 'Cash'] += position.loc[date, ticker] * sell_price
                no_played_etfs -= 1
                position.loc[date, ticker] = np.nan

        # Check if any ticker is in the Improving category (Code: 4)
        improving_tickers = Quadrant_df.loc[date, Quadrant_df.loc[date] == selected_rrg_Quadrant].index

        # Find the improving tickers from the previous day without a position
        improving_tickers_without_position = [ticker for ticker in improving_tickers if pd.isna(position.loc[date, ticker])]

        # Buy logic
        if no_played_etfs < no_etfs_selected:
            # Choose tickers based on distance_df among improving_tickers_without_position
            eligible_tickers = [ticker for ticker in improving_tickers_without_position if price_to_stop_df.loc[date, ticker] > 0]
            if tieDecider == 'MaxDistOrigo':
                chosen_tickers = distance_df.loc[date, eligible_tickers].nlargest(no_etfs_selected - no_played_etfs).index
            elif tieDecider == 'MaxDistStopPrice':
                chosen_tickers = price_to_stop_df.loc[date, eligible_tickers].nlargest(no_etfs_selected - no_played_etfs).index

            for ticker in chosen_tickers:
                if trade_type == 'MOC':
                    buy_price = adj_close_df.iloc[i][ticker]
                elif trade_type == 'OPG':
                    if i < (len(Quadrant_df) - 1):
                        buy_price = adj_open_df.iloc[i+1][ticker]
                    else:
                        buy_price = adj_close_df.iloc[i][ticker]
                # Calculate the number of shares to buy
                shares_to_buy = int(portfolio_df.loc[date, 'Portfolio Value'] / ( no_etfs_selected * buy_price))

                # Update position and cash
                position.loc[date, ticker] = shares_to_buy
                portfolio_df.loc[date, 'Cash'] -= shares_to_buy * buy_price
                no_played_etfs += 1

        # print(f"Date: {date}, Portfolio Value: {portfolio_df.loc[date, 'Portfolio Value']}, Cash: {portfolio_df.loc[date, 'Cash']}")

    return portfolio_df, position

def runBackTest(ticker_list, data_start_date, data_end_date, benchmark, atr_length, stop_loss_multiplicator, initial_pv, no_etfs_selected, tieDecider, bt_res_strat_date, wilderStopLossType, sellTypeQ, sellTypeSl, selected_rrg_Quadrant, exit_rrg_Quadrant, trade_type):
    adjCloses, adjHighs, adjLows, adjOpens = download_price_data(ticker_list, data_start_date, data_end_date)
    relStrength, relMom = calculate_rrg( benchmark, adjCloses, data_start_date, data_end_date)
    Quadrant_df = calculate_rrg_Quadrant(relStrength, relMom)
    atr_df = calculate_atr(adjCloses, adjHighs, adjLows, atr_length)
    atr_stop_df = calculate_atr_stop_loss_price(adjCloses, atr_df, atr_length, stop_loss_multiplicator, wilderStopLossType)
    price_to_stop = calculate_price_to_stop_ratio_df(adjCloses, atr_stop_df)
    distance_df = calculate_distance_from_origo_df(relStrength, relMom)
    portfolio_df, position = manage_portfolio(initial_pv, no_etfs_selected, Quadrant_df, distance_df, adjCloses, adjOpens, price_to_stop, tieDecider, sellTypeQ, sellTypeSl, selected_rrg_Quadrant, exit_rrg_Quadrant, trade_type)
    # print(Quadrant_df)
    # relStrength.to_csv('D:\Temp\RelStr.csv')
    # relMom.to_csv('D:\Temp\RelMom.csv')
    # Quadrant_df.to_csv('D:\Temp\QuadrantDf.csv')
    # atr_df.to_csv('D:\Temp\AtrDf.csv')
    # atr_stop_df.to_csv('D:\Temp\AtrStopDf.csv')
    # price_to_stop.to_csv('D:\Temp\PriceToStopDf.csv')
    # distance_df.to_csv('D:\Temp\DistancepDf.csv')
    # portfolio_df.to_csv('D:\Temp\PortfolioDf.csv')
    # position.to_csv('D:\Temp\PositionDf.csv')

    search_date = pd.to_datetime(bt_res_strat_date)
    if search_date in portfolio_df.index:
        index_location = portfolio_df.index.get_loc(search_date)
    else:
        nearest_date = portfolio_df.index[portfolio_df.index > search_date].min()
        index_location = portfolio_df.index.get_loc(nearest_date)
    bt_res_df = portfolio_df[index_location-1:].copy()
    bt_res_df['Portfolio Value'] = bt_res_df['Portfolio Value'] / bt_res_df['Portfolio Value'].iloc[0]
    bt_res_df['Cash'] = bt_res_df['Cash'] / bt_res_df['Cash'].iloc[0]

    cpa.performance_indicators(bt_res_df['Portfolio Value'])
    return

In [ ]:
ticker_list = ['IAI','IAK','IAT','IBB','IEO','IEZ','IGM','IGN','IGV','IHE','IHF','IHI','ITA','ITB','IYC','IYG','IYJ','IYT','IYZ','SOXX','XLB','XLC','XLE','XLF','XLI','XLK','XLP','XLRE','XLU','XLV','XLY']
data_start_date = '2000-01-01' # Used for YF download. Needless to change. 
data_end_date = datetime.date.today() # Used for YF download. If last PV seems weird, set end_date = today - 1 or end_date = '2023-12-12'
bt_res_start_date = '2013-01-01' # Start date of backtest.
initial_pv = 1000000

# Original paper parameters
trade_type = 'OPG' # 'MOC' or 'OPG' (original paper uses next day's open)
benchmark = 'BIL' # BIL in study. Used for RRG origin. Luby uses SPY for this.
no_etfs_selected = 5
rrg_buy_quadrant = 4 # Leading: 1, Weakening: 2, Lagging: 3 and Improving: 4
atr_length = 5
stop_loss_atr_multiplicator = 2.5 # Multiplayer on ATR to determine stop-loss price.
tieDecider = 'MaxDistStopPrice' # Which one to buy when tie: distance from origo 'MaxDistOrigo' or distance from Wilder stop-loss 'MaxDistStopPrice'. We assume that 'MaxDistStopPrice' is used in the original paper.
wilderStopLossType = 1 #  Is it allowed for the stop-loss to decrease? 0 - yes, 1 - no
sellTypeQ = 'None' # 'OutOfQuadrant', 'ReachSellQuadrant', 'None'. 'None' is used in the original paper.
sellTypeSl = 1 # 1:Use stop-loss, 0: Do not use stop-loss. Do not select 0, if sellTypeQ = 'None'(otherwise original positions will not be sold)!
rrg_sell_quadrant = 2 # Leading: 1, Weakening: 2, Lagging: 3 and Improving: 4

# Recommended parameters by us
# trade_type = 'MOC' # 'MOC' or 'OPG' (original paper uses next day's open)
# benchmark = 'SPY' # BIL in study. Used for RRG origin. Luby uses SPY for this.
# no_etfs_selected = 5
# rrg_buy_quadrant = 4 # Leading: 1, Weakening: 2, Lagging: 3 and Improving: 4
# atr_length = 5
# stop_loss_atr_multiplicator = 2.5 # Multiplayer on ATR to determine stop-loss price.
# tieDecider = 'MaxDistStopPrice' # Which one to buy when tie: distance from origo 'MaxDistOrigo' or distance from Wilder stop-loss 'MaxDistStopPrice'. We assume that 'MaxDistStopPrice' is used in the original paper.
# wilderStopLossType = 1 #  Is it allowed for the stop-loss to decrease? 0 - yes, 1 - no
# sellTypeQ = 'ReachSellQuadrant' # 'OutOfQuadrant', 'ReachSellQuadrant', 'None'. 'None' is used in the original paper.
# sellTypeSl = 1 # 1:Use stop-loss, 0: Do not use stop-loss. Do not select 0, if sellTypeQ = 'None'(otherwise original positions will not be sold)!
# rrg_sell_quadrant = 2 # Leading: 1, Weakening: 2, Lagging: 3 and Improving: 4

# Further idea: Wilder Volatility stop-loss 3 types: buy price, significant high in last atr_length day (used here now), significant high since purchase

In [ ]:
runBackTest(ticker_list, data_start_date, data_end_date, benchmark, atr_length, stop_loss_atr_multiplicator, initial_pv, no_etfs_selected, tieDecider, bt_res_start_date, wilderStopLossType, sellTypeQ, sellTypeSl, rrg_buy_quadrant, rrg_sell_quadrant, trade_type)